In [40]:
import pandas as pd

df = pd.read_csv("train.csv")


In [41]:
X = df.iloc[:,1:-1]
y = df.iloc[:,-1]

In [42]:
for i in range(199):
	X.iloc[:,i] = (X.iloc[:,i] - X.iloc[:,i].mean()) / X.iloc[:,i].std()

X.iloc[:, 199:2247] = (X.iloc[:, 199:2247] - X.iloc[:, 199:2247].mean()) / X.iloc[:, 199:2247].std()
X.iloc[:, 2247:] = (X.iloc[:, 2247:] - X.iloc[:, 2247:].mean()) / X.iloc[:, 2247:].std()

In [43]:
def kappa_cohen(y_test, y_pred):
	t = [0,0,0,0]
	l = [(1,1),(1,0),(0,1),(0,0)]

	for i in range(len(y_test)):
		t[l.index((y_test[i], y_pred[i]))] += 1

	tot = len(y_test)
	p_acc = (t[0] + t[3]) / tot
	p_has = ((t[0] + t[1]) * (t[0] + t[2]) + (t[2] + t[3]) * (t[1] + t[3])) / tot**2
	return (p_acc - p_has) / (1 - p_has)

In [44]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from lightgbm import LGBMClassifier

# Division des données
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Initialisation du modèle
def LGBM(X_train, y_train, X_test, range):
	X_train = X_train.iloc[:, range]
	X_test = X_test.iloc[:, range]

	model = LGBMClassifier(
		boosting_type="dart",
		class_weight="balanced",
		colsample_bytree=0.9408719209255059,
		learning_rate=0.9874010014614527,
		min_child_samples=30,
		min_child_weight=0.004092554311293762,
		n_estimators=904,
		n_jobs=4,
		num_leaves=64,
		random_state=33,
		reg_alpha=0.720249010918679,
		reg_lambda=0.9322794252081102,
		subsample=0.01)

	# Entraînement
	model.fit(X_train, y_train)

	# Prédictions
	return model.predict(X_test)

# Évaluation
#y_pred = LGBM(X_train, y_train, X_test, slice(-1))

In [45]:
#ranges = (slice(0, 200), slice(200, 1224), slice(1224, 2247), slice(2247,3271), slice(3271,-1))
ranges = (slice(0, 200), slice(200, 2247), slice(2247, -1))

y_preds = tuple(LGBM(X_train, y_train, X_test, r) for r in ranges)

[LightGBM] [Info] Number of positive: 3845, number of negative: 3687
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.011327 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 21559
[LightGBM] [Info] Number of data points in the train set: 7532, number of used features: 175
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0.000000
[LightGBM] [Info] Start training from score -0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain

In [46]:
y_pred = [int(y_preds[0][i] + y_preds[1][i] + y_preds[2][i] >= len(ranges) / 2.) for i in range(len(y_preds[0]))]

In [47]:
print("kappa_cohen, en prenant le plus fréquent:", kappa_cohen(y_test.array, y_pred))
print(f"Accuracy: {accuracy_score(y_test, y_pred):.2f}")
#print(classification_report(y_test, y_pred))

kappa_cohen, en prenant le plus fréquent: 0.6672139894807868
Accuracy: 0.83


In [48]:
for i in range(len(ranges)):
	print ("kappa_cohen for range", i, ":", kappa_cohen(y_test.array, y_preds[i]))

kappa_cohen for range 0 : 0.6250102252142218
kappa_cohen for range 1 : 0.6490341222944299
kappa_cohen for range 2 : 0.6405258585924766


In [78]:

def LGBM(X_train, y_train, X_test, range):
	X_train = X_train.iloc[:, range]
	X_test = X_test.iloc[:, range]

	# paramètres calculés par watson AI

	model = LGBMClassifier(
		boosting_type="dart",
		class_weight="balanced",
		colsample_bytree=0.9408719209255059,
		learning_rate=0.1,
		min_child_samples=30,
		min_child_weight=0.004092554311293762,
		n_estimators=504,
		n_jobs=4,
		num_leaves=64,
		random_state=33,
		reg_alpha=0.720249010918679,
		reg_lambda=0.9322794252081102,
		subsample=0.01,
		verbose = -1)

	model.fit(X_train, y_train)

	return model.predict_proba(X_test)

# on calcule le résultat majoritaire entre celui donné en se basant sur les 200 premières features, les 2048 suivantes et les 2048 suivantes

ranges = (slice(0, 200), slice(200, 2247), slice(2247, -1))

def compute():
	X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

	y_preds = tuple(LGBM(X_train, y_train, X_test, r) for r in ranges)

	# on prend 1 si plus de la moitié des résultats vaut 1
	y_pred = [int(y_preds[0][i][1] + y_preds[1][i][1] + y_preds[2][i][1] >= 1.5) for i in range(len(y_preds[0]))]

	print("kappa_cohen total, en prenant le plus fréquent:", kappa_cohen(y_test.array, y_pred))

	for i in range(len(ranges)):
		print ("kappa_cohen avec le range", i, ":", kappa_cohen(y_test.array, list(map(lambda t: int(t[1] >= 0.5), y_preds[i]))))

compute()
#y_pred = compute()

kappa_cohen total, en prenant le plus fréquent: 0.6546765309824797
kappa_cohen avec le range 0 : 0.6345178810622449
kappa_cohen avec le range 1 : 0.6122185065592703
kappa_cohen avec le range 2 : 0.6196436087798944


In [74]:
kappa_cohen(y_test.array, y_pred)

0.6725191503490147

array([[3.09648967e-02, 9.69035103e-01],
       [2.51144050e-01, 7.48855950e-01],
       [9.85879728e-01, 1.41202720e-02],
       ...,
       [9.85399514e-01, 1.46004858e-02],
       [6.87573340e-02, 9.31242666e-01],
       [9.99402133e-01, 5.97867498e-04]])

In [64]:
sum(y_preds[0][0])

1.0

In [66]:
c = 0
for t in y_preds[0]:
	if 0.3 < t[0] < 0.7:
		c += 1

print (c / len(y_preds[0]))

0.09028146574614976


In [81]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
X_train.drop(columns=["BalabanJ"])

,BalabanJ,BertzCT,Chi0,Chi0n,Chi0v,Chi1,Chi1n,Chi1v,Chi2n,Chi2v,...,fcfc_2038,fcfc_2039,fcfc_2040,fcfc_2041,fcfc_2042,fcfc_2043,fcfc_2044,fcfc_2045,fcfc_2046,fcfc_2047
873,-0.949184,2.002450,2.361105,1.854961,1.708121,2.007077,1.750090,1.482624,1.836141,1.460753,...,-0.043764,-0.789093,-0.369726,-0.37146,-0.084655,-0.053626,1.711758,-0.148047,-0.030931,-0.089001
2742,2.087643,-1.719548,-1.923964,-2.004633,-1.919699,-2.079808,-2.206259,-2.170786,-2.087660,-2.024442,...,-0.043764,0.213648,-0.369726,-0.37146,-0.084655,-0.053626,-0.482415,-0.148047,-0.030931,-0.089001
2995,0.711589,-0.330186,-0.907052,-0.844336,-0.762685,-0.838090,-0.764537,-0.776889,-0.661718,-0.667489,...,-0.043764,1.216390,-1.134607,-0.37146,-0.084655,-0.053626,-0.482415,-0.148047,-0.030931,-0.089001
2178,0.842471,0.255090,0.726022,0.368837,0.928109,0.654305,0.223706,0.766721,-0.128909,0.503502,...,-0.043764,0.213648,-0.369726,-0.37146,-0.084655,-0.053626,-0.482415,-0.148047,-0.030931,-0.089001
8168,-0.146332,-0.146428,-0.181711,-0.575120,-0.715082,-0.215700,-0.620569,-0.809398,-0.748047,-0.971380,...,-0.043764,1.216390,1.924915,-0.37146,-0.084655,-0.053626,0.614671,-0.148047,-0.030931,-0.089001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4878,0.606307,0.384646,-0.101441,-0.031464,0.065581,-0.008630,0.005954,0.655570,0.000965,0.920032,...,-0.043764,-0.789093,0.395154,-0.37146,-0.084655,-0.053626,0.614671,-0.148047,-0.030931,-0.089001
9373,-0.082960,-0.684536,-1.529156,-1.477561,-1.614969,-1.276307,-1.365151,-1.529281,-1.474015,-1.654631,...,-0.043764,-0.789093,1.924915,-0.37146,-0.084655,-0.053626,-0.482415,-0.148047,-0.030931,-0.089001
5736,-0.140163,-0.211546,-0.570062,-0.281821,-0.422613,-0.487204,-0.213479,-0.415811,-0.320387,-0.568884,...,-0.043764,-0.789093,0.395154,-0.37146,-0.084655,-0.053626,-0.482415,-0.148047,-0.030931,-0.089001
6899,-0.632793,-0.465688,-0.709648,-0.515915,-0.417499,-0.436340,-0.183590,-0.015997,-0.237289,0.024100,...,-0.043764,1.216390,-1.134607,-0.37146,-0.084655,-0.053626,0.614671,-0.148047,-0.030931,-0.089001


In [94]:
from sklearn.feature_selection import VarianceThreshold

selector = VarianceThreshold(threshold = 0.01)
selector.fit(X_train)

X_t2 = pd.DataFrame(X_train, columns = X_train.columns[selector.get_support()])

In [96]:
tuple(i + j for i in range(3) for j in range(4))

(0, 1, 2, 3, 1, 2, 3, 4, 2, 3, 4, 5)

In [97]:
sum(i for i in range(5))

10

In [100]:
np.array(tuple(i for i in range(5) if i % 2))

array([1, 3])

In [110]:
df_train

NameError: name 'df_train' is not defined

In [111]:
df

,smiles,BalabanJ,BertzCT,Chi0,Chi0n,Chi0v,Chi1,Chi1n,Chi1v,Chi2n,...,fcfc_2039,fcfc_2040,fcfc_2041,fcfc_2042,fcfc_2043,fcfc_2044,fcfc_2045,fcfc_2046,fcfc_2047,class
0,Brc1cc(Br)c(Oc2cc(Br)c(Br)cc2Br)cc1Br,2.415403,585.102114,14.154336,8.985436,18.501415,8.879918,4.851543,9.609532,3.495540,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,Brc1ccc2[nH]c3c(c2c1)CCCC3Nc1ccccc1,1.756595,776.517592,14.233840,11.695437,13.281434,10.309663,7.426852,8.219850,5.662312,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,Brc1ccc2c(NC3=NCC4(CN5CCC4CC5)O3)ncnn12,1.393870,796.361052,15.440947,12.574323,14.160319,11.233913,7.945929,8.738927,6.525386,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,Brc1cnc2nc(N3CCN4CCC3CC4)oc2c1,1.724265,605.709066,12.819626,10.549759,12.135755,9.292826,6.705489,7.498487,5.237927,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,Brc1cncc(COCC2(c3ccccc3)CCNCC2)c1,1.684628,597.045968,15.252866,12.594869,14.180866,10.784424,7.719435,8.512433,5.900562,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9410,c1csc(-c2ccc(OCCCN3CCCCC3)cc2)c1,1.408350,514.661172,14.330216,12.462016,13.278513,10.415816,7.908018,8.787671,5.605995,...,1.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,1
9411,c1nc(NC2CCC(N3CCOCC3)CC2)c2c(C3CCOCC3)c[nH]c2n1,1.281401,774.193965,18.760938,16.530170,16.530170,13.882143,10.863157,10.863157,8.374672,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
9412,c1ncc(-c2cc3sc(N4CCC(N5CCCCC5)CC4)nc3cn2)cn1,1.307013,897.235753,18.053831,15.419593,16.236089,13.365306,9.876652,10.693149,7.475712,...,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
9413,c1ncc(-c2ccc(OCCCN3CCCCC3)cc2)o1,1.408350,516.232953,14.330216,12.331880,12.331880,10.415816,7.757749,7.757749,5.494375,...,2.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,1


In [124]:
cols = (df.columns[1:200], df.columns[200:2248], df.columns[2248:-1])


In [125]:
df1 = df.drop(columns = cols[1].union(cols[2]))
df2 = df.drop(columns = cols[0].union(cols[2]))
df3 = df.drop(columns = cols[0].union(cols[1]))

In [129]:
df1.to_csv("train_1.csv", index=False)
df2.to_csv("train_2.csv", index=False)
df3.to_csv("train_3.csv", index=False)

In [117]:
df.columns[2248:]

Index(['fcfc_0000', 'fcfc_0001', 'fcfc_0002', 'fcfc_0003', 'fcfc_0004',
       'fcfc_0005', 'fcfc_0006', 'fcfc_0007', 'fcfc_0008', 'fcfc_0009',
       ...
       'fcfc_2039', 'fcfc_2040', 'fcfc_2041', 'fcfc_2042', 'fcfc_2043',
       'fcfc_2044', 'fcfc_2045', 'fcfc_2046', 'fcfc_2047', 'class'],
      dtype='object', length=2049)

In [131]:
y_train.co

7532

In [133]:
df = pd.read_csv("../pred_2.csv")

In [139]:
for i in range(1, 7):
	print ("Confiance pour la série", i, df[df["series"] == "series_"+str(i)]["probas"].mean())

Confiance pour la série 1 0.7445782092608775
Confiance pour la série 2 0.7575045200995351
Confiance pour la série 3 0.5841255058891401
Confiance pour la série 4 0.6015639837759355
Confiance pour la série 5 0.696059711063312
Confiance pour la série 6 0.776755307542385


In [138]:
df[df["series"] == "series_1"]["probas"].mean()

0.7445782092608775

In [135]:
d1 = pd.read_csv()

Index(['smiles', 'class', 'series', 'probas'], dtype='object')

In [140]:
df = pd.read_csv("train_cluster.csv")
d1.

In [162]:
df.loc[0,"series_0"]

False

In [174]:
def add_series(nom):
	df = pd.read_csv(nom)
	for i in range(13):
		if "series_"+str(i) not in df.columns:
			df["series_"+str(i)] = [0] * len(df)
	for i in range(13):
		df["series_"+str(i)] = df["series_"+str(i)].astype(float)
	if "Unnamed: 0.1" in df.columns:
		df = df.drop(columns = ["Unnamed: 0.1"])
		print ("a")
	if "Unnamed: 0" in df.columns:
		df = df.drop(columns = ["Unnamed: 0"])
		print ("b")
	df.to_csv(nom, index = False)

add_series("../data/test_1_cluster.csv")
add_series("../data/test_2_cluster.csv")


b
b


In [175]:
df1 = pd.read_csv("../data/test_1_cluster.csv")
df1.columns

Index(['smiles', 'BalabanJ', 'BertzCT', 'Chi0', 'Chi0n', 'Chi0v', 'Chi1',
       'Chi1n', 'Chi1v', 'Chi2n',
       ...
       'series_4', 'series_5', 'series_6', 'series_7', 'series_9', 'series_10',
       'series_11', 'series_12', 'series_0', 'series_8'],
      dtype='object', length=4309)

In [170]:
df1 = 

Index(['Unnamed: 0.1', 'Unnamed: 0', 'smiles', 'BalabanJ', 'BertzCT', 'Chi0',
       'Chi0n', 'Chi0v', 'Chi1', 'Chi1n',
       ...
       'series_4', 'series_5', 'series_6', 'series_7', 'series_9', 'series_10',
       'series_11', 'series_12', 'series_0', 'series_8'],
      dtype='object', length=4311)

In [169]:
df1.head()

,Unnamed: 0.1,Unnamed: 0,smiles,BalabanJ,BertzCT,Chi0,Chi0n,Chi0v,Chi1,Chi1n,...,series_4,series_5,series_6,series_7,series_9,series_10,series_11,series_12,series_0,series_8
0,0,0,Brc1cccc2nc(C#Cc3ccccn3)ccc12,2.013094,785.948261,13.079140,9.968544,11.554541,9.326500,5.811377,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,1,C#CC1(O)CCC2C3CCC4=CC(=O)CCC4(C)C3CCC21C,1.702410,626.243053,16.457819,14.399067,14.399067,10.839101,9.280285,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,2,C#CC1CCC(C#N)N1C(=O)CNC1(C)CCN(c2cc(C(=O)O)ccn...,1.564182,841.802921,21.018297,16.466414,16.466414,13.864322,9.729429,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,3,C#CCN(C)CC1=Cc2ccccc2Oc2ccccc21,2.066873,724.341367,14.656490,12.043178,12.043178,10.275188,6.965750,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,4,C#Cc1cc(-c2n[nH]c3c2Cc2cc(CN4CCN(C)CC4)ccc2-3)cs1,1.389134,1031.007844,18.380104,15.456631,16.273128,13.207841,9.368787,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [155]:
dff.iloc[:, 4296:4309]

,series_0,series_1,series_2,series_3,series_4,series_5,series_6,series_7,series_8,series_9,series_10,series_11,series_12
0,False,True,False,False,False,False,False,False,False,False,False,False,False
1,False,False,True,False,False,False,False,False,False,False,False,False,False
2,False,True,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,True,False,False,False,False,False,False,False,False,False
4,False,True,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9410,False,False,False,False,False,False,False,False,False,False,False,False,True
9411,False,False,False,True,False,False,False,False,False,False,False,False,False
9412,False,False,False,True,False,False,False,False,False,False,False,False,False
9413,False,False,False,False,False,False,False,False,False,False,True,False,False
